In [ ]:
!pip install keras-rl2
!pip install 'gym[all]'

  Using cached https://files.pythonhosted.org/packages/cf/8c/64e0630b3d450244feef0688d90eab2448631e40ba6bdbd90a70b84898e7/mujoco-py-1.50.1.68.tar.gz
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-psyzy_hd/mujoco-py/setup.py'"'"'; __file__='"'"'/tmp/pip-install-psyzy_hd/mujoco-py/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-9pb2d6xg/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [ ]:
%%bash

# install required system dependencies
sudo apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists...
Building dependency tree...
Reading state information...
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [ ]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [ ]:
import numpy as np

import gym
from gym.envs.registration import registry, register, make, spec

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, Reshape
from tensorflow.keras.optimizers import Adam


from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from car_racing_v1 import *

In [ ]:
register(
    id='CarRacing-v1',
    entry_point='gym.envs.box2d:CarRacing',
    max_episode_steps=2000,
    reward_threshold=900,
)

In [ ]:
# Get the environment and extract the number of actions.
ENV_NAME = 'CarRacing-v1'
env = CarRacing()

nb_actions = len(env.action_space)
input_shape = env.observation_space.shape

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
print("nb actions = ", len(env.action_space))
print("observation_space.shape = ", input_shape)

nb actions =  16
observation_space.shape =  (96, 96, 3)


In [ ]:
def build_model(input_shape, nb_actions):
    model = Sequential()
    print(input_shape)
    
    model.add(Reshape(input_shape, input_shape = (1,96,96,3)))
    model.add(Convolution2D(32, (8, 8)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    
    
    '''model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))'''
    
    print(model.summary())
    return model

In [ ]:
model = build_model(input_shape, nb_actions)

(96, 96, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 96, 96, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 89, 89, 32)        6176      
_________________________________________________________________
activation (Activation)      (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 86, 86, 64)        32832     
_________________________________________________________________
activation_1 (Activation)    (None, 86, 86, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 84, 84, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 84, 84, 

In [ ]:
def build_agent(model, nb_actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=10000)
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=nb_actions, nb_steps_warmup=500, target_model_update=100)
    return dqn

In [ ]:
dqn = build_agent(model, nb_actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
weights_filename = 'dqn_' + ENV_NAME + '_weights.h5f'
checkpoint_weights_filename = 'dqn_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = 'dqn_' + ENV_NAME + '_log.json'
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=2500)]
callbacks += [FileLogger(log_filename, interval=100)]

In [ ]:
import os
if os.path.exists(weights_filename + '.data-00000-of-00001') and os.path.exists(weights_filename + '.index'):
  dqn.load_weights(weights_filename)
  print('Weights loaded')
else:
  print('Weights not found')

Weights loaded


In [ ]:
dqn.fit(env, nb_steps=3000, verbose=2, nb_max_episode_steps=200, action_repetition=3, visualize = False)

Training for 3000 steps ...
Track generation: 1187..1488 -> 301-tiles track


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   18/3000: episode: 1, duration: 8.003s, episode steps:  18, steps per second:   2, episode reward: -95.200, mean reward: -5.289 [-100.100,  6.367], mean action: 6.944 [0.000, 14.000],  loss: --, mae: --, mean_q: --, mean_eps: --
Track generation: 1107..1388 -> 281-tiles track
   36/3000: episode: 2, duration: 0.506s, episode steps:  18, steps per second:  36, episode reward: -94.486, mean reward: -5.249 [-100.100,  6.843], mean action: 8.556 [0.000, 14.000],  loss: --, mae: --, mean_q: --, mean_eps: --
Track generation: 1156..1455 -> 299-tiles track
   54/3000: episode: 3, duration: 0.483s, episode steps:  18, steps per second:  37, episode reward: -95.133, mean reward: -5.285 [-100.100,  6.411], mean action: 7.556 [1.000, 15.000],  loss: --, mae: --, mean_q: --, mean_eps: --
Track generation: 1160..1455 -> 295-tiles track
   72/3000: episode: 4, duration: 0.529s, episode steps:  18, steps per second:  34, episode reward: -94.996, mean reward: -5.278 [-100.100,  6.503], mean action: 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  503/3000: episode: 28, duration: 6.847s, episode steps:  18, steps per second:   3, episode reward: -95.523, mean reward: -5.307 [-100.100,  6.152], mean action: 5.944 [0.000, 15.000],  loss: 71.524910, mae: 44.471125, mean_q: -43.849440, mean_eps: 0.954865
Track generation: 1131..1418 -> 287-tiles track
  521/3000: episode: 29, duration: 5.076s, episode steps:  18, steps per second:   4, episode reward: -94.710, mean reward: -5.262 [-100.100,  6.693], mean action: 5.278 [0.000, 15.000],  loss: 120.545693, mae: 44.106391, mean_q: -43.596812, mean_eps: 0.953965
Track generation: 1059..1328 -> 269-tiles track
  539/3000: episode: 30, duration: 5.085s, episode steps:  18, steps per second:   4, episode reward: -90.375, mean reward: -5.021 [-100.200,  7.163], mean action: 5.556 [0.000, 14.000],  loss: 76.274684, mae: 43.859257, mean_q: -42.906503, mean_eps: 0.952345
Track generation: 1134..1426 -> 292-tiles track
retry to generate track (normal if there are not manyinstances of this mess

In [ ]:
 # After training is done, we save the final weights one more time.
dqn.save_weights(weights_filename, overwrite=True)

In [ ]:
# Finally, evaluate our algorithm for 10 episodes.
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Track generation: 1286..1614 -> 328-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1252..1569 -> 317-tiles track
Episode 1: reward: -86.513, steps: 55
Track generation: 1296..1624 -> 328-tiles track
Episode 2: reward: -87.151, steps: 55
Track generation: 1148..1444 -> 296-tiles track
Episode 3: reward: -85.161, steps: 55
Track generation: 1084..1359 -> 275-tiles track
Episode 4: reward: -83.602, steps: 55
Track generation: 1256..1574 -> 318-tiles track
Episode 5: reward: -86.573, steps: 55
Track generation: 1094..1377 -> 283-tiles track
Episode 6: reward: -84.223, steps: 55
Track generation: 1210..1516 -> 306-tiles track
Episode 7: reward: -85.828, steps: 55
Track generation: 1268..1589 -> 321-tiles track
Episode 8: reward: -86.750, steps: 55
Track generation: 1124..1409 -> 285-tiles track
Episode 9: reward: -84.373, steps: 55
Track generation: 1206..1523 -> 317-tiles track
Episode 10: reward: -8

In [ ]:
for i in range(10):
  dqn.load_weights(weights_filename)
  dqn.fit(env, nb_steps=3000, verbose=2, nb_max_episode_steps=200, action_repetition=3, visualize = False)
  dqn.save_weights(weights_filename, overwrite=True)

Training for 3000 steps ...
Track generation: 1004..1262 -> 258-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1163..1462 -> 299-tiles track
   18/3000: episode: 1, duration: 0.784s, episode steps:  18, steps per second:  23, episode reward: -95.133, mean reward: -5.285 [-100.100,  6.411], mean action: 7.667 [0.000, 15.000],  loss: --, mae: --, mean_q: --, mean_eps: --
Track generation: 1208..1514 -> 306-tiles track
   36/3000: episode: 2, duration: 0.488s, episode steps:  18, steps per second:  37, episode reward: -95.364, mean reward: -5.298 [-100.100,  6.257], mean action: 7.111 [1.000, 15.000],  loss: --, mae: --, mean_q: --, mean_eps: --
Track generation: 1216..1524 -> 308-tiles track
   54/3000: episode: 3, duration: 0.492s, episode steps:  18, steps per second:  37, episode reward: -95.428, mean reward: -5.302 [-100.100,  6.215], mean action: 6.833 [0.000, 15.000],  loss: --, mae: --, mean_q: --, mean_eps: --
Track g